In [ ]:
#Проект чат-бот который считает % проверенных домашних заданий педагогами и отправляет автоматическое сообщение педагогу, если % проверки ниже 75%

In [1]:
import telebot #Библеотека для работы с чат-ботом телеграм
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [ ]:
# Установите свои учетные данные для Google Sheets
SCOPE = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive.file",
    "https://www.googleapis.com/auth/drive"
]

# Замените на путь к вашему файлу credentials.json
CREDENTIALS_FILE = "C:/Users/1/Desktop/Project (Bot)/projectbot-447014-437b4c89d4a6.json"
credentials = ServiceAccountCredentials.from_json_keyfile_name(CREDENTIALS_FILE, SCOPE)

# Подключение к Google Sheets
client = gspread.authorize(credentials)
spreadsheet_name = "Отчёт по дз"
sheet = client.open(spreadsheet_name).sheet1

# Создание бота
bot = telebot.TeleBot('7809768765:AAHOm3QIEj-TEE-XTWZ9_Y8R0Tn0rzK8hwk')

@bot.message_handler(commands=['start'])
def start(message):
    bot.send_message(message.chat.id, f'Привет, {message.from_user.first_name} {message.from_user.last_name}!')

@bot.message_handler(commands=['report'])
def report(message):
    try:
        # Получение данных из таблицы
        data = sheet.get_all_values()
        
        # Форматирование данных для отправки
        formatted_data = "\n".join(["\t".join(row) for row in data])
        bot.send_message(message.chat.id, f"Данные из таблицы:\n{formatted_data}")
    except Exception as e:
        bot.send_message(message.chat.id, f"Ошибка при чтении данных: {e}")

bot.polling(none_stop=True)